G= (T, N, R,S)

Conjunto de terminales: T= {time, flies, arrow, an, like}

Conjunto de no terminales: N= {S, NP, VP, PP, Det, Nominal, Verb, Preposition, Noun}

Conjunto de reglas:

 R={

 S → NP VP				0.800

 NP → time | flies | arrow		0.002

 NP → Det Nominal			0.300

 NP → Nominal Nominal		0.200

 Nominal → time | flies | arrow	0.002

 Nominal → Nominal Noun		0.100

 Nominal → Nominal PP		0.200

 VP → time				0.004

 VP → flies| like			0.008

 VP → Verb NP				0.300

 VP → Verb PP				0.200

 PP → Preposition NP			0.100

 Verb → time				0.010

 Verb → flies| like			0.020

 Noun → time | flies | arrow		0.010

 Det → an				0.050

 Preposition → like			0.050

 }
 

In [1]:
file_grammar = """
S -> NP VP
NP -> Det Nominal
NP -> Nominal Nominal
Nominal -> Nominal Noun
Nominal -> Nominal PP
VP -> Verb NP
VP -> Verb PP
PP -> Preposition NP
"""

Se crea el lexicón, de forma común, en lingüística se dice que un elemento del lexicón en la entrada de un diccionario, entonces se define como tal en el código.

In [2]:
lexicon_dictionary = {
    'NP': {'time', 'flies', 'arrow'},
    'Nominal': {'time', 'flies', 'arrow'},
    'VP': {'time', 'flies', 'like'},
    'Verb': {'time', 'flies', 'like'},
    'Noun': {'time', 'flies', 'arrow'},
    'Det': {'an'},
    'Preposition': {'like'}
}

In [3]:
def grammar_rules(grammar_buffer):
    rules = []
    # leer cada línea del texto
    for line in grammar_buffer.strip().split("\n"):
        # separar cada línea en sus dos miembros
        parent, chidren = line.strip().split("->")
        # Quitar espacio en blanco posterior
        parent = parent.strip()
        # Distinguir los hijos
        chidren = chidren.split()
        #crear la regla (parent, (c1, c2)) como en lisp ;)
        rule = (parent, tuple(chidren))
        rules.append(rule)
    # Crear un diccionario con las reglas invertidas, eso permite buscar al nodo padre
    find_parent = dict()
    for parent, (lc, rc) in rules:
        # Si no existe la entrada, se crea
        if (lc, rc) not in find_parent:
            find_parent[lc, rc] = []
        # Se agrega el caso
        find_parent[lc,rc].append(parent)
    return find_parent

#parents = set(val[0] for val in list(find_parent.values()))
#for i in list(lexicon.keys()):
#    parents.add(i)


In [4]:
def cky_parser(sentence, grammar, lexicon, verbose=False):
    words = sentence.lower().split()
    if verbose:
        print(words)
    matrix_cky = {}
    for i in range(len(words)):
        # Las ordenadas van del 0 al 4
        ordinate = 0
        if verbose:
            print("0: i=",i)
        # Las abscisas van de 1 a 5
        for abscissa in range(i+1, len(words)+1):
            # Se agrega una celda
            matrix_cky[(ordinate, abscissa)] = []
            if verbose:
                print("1: abscissa=",abscissa,"ordinate=",ordinate,"a-o=",abscissa-ordinate)
            # Si se opera sobre la diagonal
            if(abscissa-ordinate==1):
                # Combina la palabra con las categorías del lexicón
                for key in lexicon_dictionary:
                    if verbose:
                        print("2: key=",key,"word=",words[ordinate])
                    # Si la combinación existe en el lexicón se agrega a la matriz
                    if(words[ordinate] in lexicon[key]):
                        matrix_cky[(ordinate,abscissa)].append((key,0,words[ordinate],words[ordinate]))
                        if verbose:
                            print("2:",matrix_cky)
            # Si hay que operar sobre dos celdas
            elif(abscissa-ordinate>1):
                if verbose:
                    print("3:","(",ordinate,",",abscissa,")")
                # Obtener los valores de las celdas
                for index in range(abscissa-ordinate-1):
                    left = matrix_cky[(ordinate,abscissa-1-index)]
                    down = matrix_cky[(abscissa-1-index,abscissa)]
                    if verbose:
                        print("4: index=",index)
                        print("4: left=",left,"\n  ","down=",down)
                    # Si una de las celdas esta vacía no se hace nada
                    if not left or not down:
                        if verbose:
                            print("5: Nothing to do")
                    else:
                        # Combinar los valores de las celdas
                        for a in left:
                            for b in down:
                                # Obtiene el primer elemento de la tupla
                                # Crea la tupla para buscarla en las reglas
                                if((a[0],b[0]) in grammar):
                                    matrix_cky[(ordinate,abscissa)].append(
                                        (grammar[(a[0],b[0])][0],
                                        abscissa-1-index,a[0],b[0])
                                        )
                                    if verbose:
                                        print("6: add=",grammar[(a[0],b[0])][0],abscissa-1-index,a[0],b[0])
                                else:
                                    if verbose:
                                        print("6: Nothing to do")
            if verbose:
                print(matrix_cky)
                print("9: Next Ordinate")
            ordinate+=1
    return matrix_cky

In [5]:
sentence = "Time flies like an arrow"
grammar = grammar_rules(file_grammar)
cky = cky_parser(sentence, grammar, lexicon_dictionary, True)

['time', 'flies', 'like', 'an', 'arrow']
0: i= 0
1: abscissa= 1 ordinate= 0 a-o= 1
2: key= NP word= time
2: {(0, 1): [('NP', 0, 'time', 'time')]}
2: key= Nominal word= time
2: {(0, 1): [('NP', 0, 'time', 'time'), ('Nominal', 0, 'time', 'time')]}
2: key= VP word= time
2: {(0, 1): [('NP', 0, 'time', 'time'), ('Nominal', 0, 'time', 'time'), ('VP', 0, 'time', 'time')]}
2: key= Verb word= time
2: {(0, 1): [('NP', 0, 'time', 'time'), ('Nominal', 0, 'time', 'time'), ('VP', 0, 'time', 'time'), ('Verb', 0, 'time', 'time')]}
2: key= Noun word= time
2: {(0, 1): [('NP', 0, 'time', 'time'), ('Nominal', 0, 'time', 'time'), ('VP', 0, 'time', 'time'), ('Verb', 0, 'time', 'time'), ('Noun', 0, 'time', 'time')]}
2: key= Det word= time
2: key= Preposition word= time
{(0, 1): [('NP', 0, 'time', 'time'), ('Nominal', 0, 'time', 'time'), ('VP', 0, 'time', 'time'), ('Verb', 0, 'time', 'time'), ('Noun', 0, 'time', 'time')]}
9: Next Ordinate
1: abscissa= 2 ordinate= 1 a-o= 1
2: key= NP word= flies
2: {(0, 1): [(

In [9]:
N = len(sentence.split())
#print(cky[0,N])
#sol = []
#for j in range(len(cky[0,N])):
#    if cky[0,N][j][0] == "S":
#        sol.append(cky[0,N][j][0])

def tree (matrix,cell = (0,N),parent="S",index=0):
    list = []
    print("1: Celda:",cell,"with index",index)
    print("1: Contenido",matrix[cell])
    print("2: if",len(matrix[cell]),"== 0")
    if len(matrix[cell]) == 0:
        print("2: No solution found")
        return list                                            
    print("3: if",matrix[cell][0][1],"== 0")
    if matrix[cell][0][1] == 0:
        print("3: if",matrix[cell][index][0],"=",parent)
        if matrix[cell][index][0] == parent:
            print("3: return",[matrix[cell][index][0], matrix[cell][index][2]])
            return [matrix[cell][index][0], matrix[cell][index][2]]
        else:
            print("3: Try next index=",index+1)
            tree(matrix,cell,parent,index+1)
    print("4: if",matrix[cell][index][0],"==",parent,"in",cell)
    elif (matrix[cell][index][0]) == parent:
        # Agrega el símbolo
        print("6: add",matrix[cell][index][0])
        list.extend(matrix[cell][index][0])
        # Inspeccionar hijos
        child = []
        # Hijo derecho
        print("7: New r tuple",(cell[0],matrix[cell][index][1]),"from",cell,"with parent",matrix[cell][index][2])
        r = tree(matrix, (cell[0],matrix[cell][index][1]),matrix[cell][index][2])
        print("7: Lista r child",r)
        child.extend(r)
        print("7: child",child)
        # Hijo izquierdo
        print("7: New i tuple",(matrix[cell][index][1],cell[1]),"from",cell,"with parent",matrix[cell][index][3])
        child.extend(
            tree(matrix,
            (matrix[cell][index][1],cell[1]),
            matrix[cell][index][3]))
        print("8: Lista l child",child)
        print("8: Integrate list")
        list.extend(child)
        print("8:", list)
    else:
        print("9: Start tree with index",index+1)
        tree(matrix,cell,parent,index+1)
    print("0: return",list)
    return list


print(tree(cky, (0,5), "S"))

SyntaxError: invalid syntax (<ipython-input-9-5bc304e959eb>, line 26)

In [ ]:
def backpointer(dict,list,tuple,parent="S"):
    print("Begin if:",tuple)
    if(len(dict[tuple])==0):
        return []
    if(dict[tuple][0][1]==0):
        print("0:",dict[tuple])
        return [dict[tuple][0][0], dict[tuple][0][2]]
    print("End if:",dict[tuple][0])
    print("Append:",dict[tuple][0][0])                            
    list.append(dict[tuple][0][0])
    print("list:",list)
    list2=[]
    print("Recursion 1:",tuple[0],",",dict[tuple][0][1])
    list2.append(backpointer(dict, [], (tuple[0],dict[tuple][0][1]),dict[tuple][0][2]))
    print("Recursion 2:",dict[tuple][0][1],",",tuple[1])
    list2.append(backpointer(dict, [], (dict[tuple][0][1],tuple[1]),dict[tuple][0][3]))
    list.append(list2)
    print("Fin:",list)
                                                        
    return list

backpointer(cky,[],(0,N))

#if cky[(0,5)][1][0] == "S":
#    print("Solución")

Begin if: (0, 5)
End if: ('S', 2, 'NP', 'VP')
Append: S
list: ['S']
Recursion 1: 0 , 2
Begin if: (0, 2)
End if: ('S', 1, 'NP', 'VP')
Append: S
list: ['S']
Recursion 1: 0 , 1
Begin if: (0, 1)
0: [('NP', 0, 'time', 'time'), ('Nominal', 0, 'time', 'time'), ('VP', 0, 'time', 'time'), ('Verb', 0, 'time', 'time'), ('Noun', 0, 'time', 'time')]
Recursion 2: 1 , 2
Begin if: (1, 2)
0: [('NP', 0, 'flies', 'flies'), ('Nominal', 0, 'flies', 'flies'), ('VP', 0, 'flies', 'flies'), ('Verb', 0, 'flies', 'flies'), ('Noun', 0, 'flies', 'flies')]
Fin: ['S', [['NP', 'time'], ['NP', 'flies']]]
Recursion 2: 2 , 5
Begin if: (2, 5)
End if: ('VP', 3, 'Verb', 'NP')
Append: VP
list: ['VP']
Recursion 1: 2 , 3
Begin if: (2, 3)
0: [('VP', 0, 'like', 'like'), ('Verb', 0, 'like', 'like'), ('Preposition', 0, 'like', 'like')]
Recursion 2: 3 , 5
Begin if: (3, 5)
End if: ('NP', 4, 'Det', 'Nominal')
Append: NP
list: ['NP']
Recursion 1: 3 , 4
Begin if: (3, 4)
0: [('Det', 0, 'an', 'an')]
Recursion 2: 4 , 5
Begin if: (4, 5)
0

['S',
 [['S', [['NP', 'time'], ['NP', 'flies']]],
  ['VP', [['VP', 'like'], ['NP', [['Det', 'an'], ['NP', 'arrow']]]]]]]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=16bbac0e-c0cf-475a-9977-7067c93e2eaf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>